<a href="https://colab.research.google.com/github/sankalp24/pytorch_tutorial/blob/master/neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
#print(params)
print(len(params))
print(params[0])  # conv1's .weight

10
Parameter containing:
tensor([[[[ 0.0699,  0.1504,  0.2788],
          [ 0.2885,  0.1590,  0.1012],
          [-0.2921,  0.1604,  0.0300]]],


        [[[-0.2717, -0.1456,  0.2601],
          [ 0.2105, -0.0905, -0.0184],
          [ 0.1171, -0.2005, -0.0287]]],


        [[[-0.1364,  0.0765, -0.2732],
          [-0.3194, -0.0322,  0.0618],
          [ 0.1121, -0.0515, -0.2312]]],


        [[[-0.0159,  0.0643,  0.3286],
          [-0.0297,  0.0367,  0.0331],
          [-0.1310, -0.2246,  0.2448]]],


        [[[-0.3024,  0.2571, -0.2058],
          [-0.1682, -0.1395, -0.0931],
          [ 0.0677,  0.1397,  0.0627]]],


        [[[-0.2123,  0.0550,  0.3133],
          [-0.2045,  0.1912,  0.2148],
          [ 0.0462, -0.1987, -0.0334]]]], requires_grad=True)


In [4]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)

tensor([[[[ 0.2326, -1.0478,  0.1373,  ...,  0.7816, -1.3431,  1.6413],
          [-0.9341, -0.0457, -0.1517,  ...,  2.3895, -0.1579, -0.7628],
          [ 0.0981, -0.6117, -2.4312,  ..., -0.7317,  1.0210,  0.2896],
          ...,
          [ 1.3058,  0.3217, -1.2373,  ...,  0.1869, -0.1296, -2.1397],
          [ 0.0313,  0.2503, -0.6496,  ...,  0.1277, -0.0641,  0.7040],
          [-0.6344,  1.7684, -0.7530,  ...,  1.9481,  0.8835,  0.8903]]]])
tensor([[-0.1264, -0.0028, -0.0123, -0.0795, -0.0469,  0.0367, -0.0363,  0.1016,
          0.1393,  0.0341]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3886, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0003, -0.0011, -0.0133,  0.0246,  0.0054,  0.0216])


In [0]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update